# Random Walks on Undirected Graphs

In this notebook, we will implement and describe a randomized algorithm that solves the s-t connectivity problem.

## Representing Graphs

We will use an adjacency list representation by means of a dictionary. Vertices will be keys and the list of edges will be values. An example is the following graph:

In [1]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E'],
    'G': []
}

Here, for example, we have a vertex *A* that has edges to the two vertices *B* and *C*. We note that by the definition of undirected graphs, if *A* is connected to *B*, then *B* is also connected to *A*, which is apparent in the representation shown above.

## The s-t Connectivity Problem

We are given an undirected graph *G = (V,E)* and two vertices *s*(=start) and t(=target) in G. We denote |V| = *n* and |E| = *m*; that is, the number of vertices is *n* and the nubmer of edges is *m*. The goal is to check wheter there exist an s-t path (i.e., wheter we can reach *t* when we start at *s*).

## A First Solution Using BFS

We can solve this problem using a graph traversal algorithm. We can either use BFS or DFS. Both of these solutions taken a linear time (O(n)). 

In [2]:
from collections import deque

def bfs(graph, start, target):
    explored = []
    
    queue = [start]
    
    levels = {}
    levels[start] = 0
    
    visited = [start]
    
    while queue:
        node = queue.pop(0)
        explored.append(node)
        neighbors = graph[node]
        
        for neighbor in neighbors:
            if neighbor not in visited:
                queue.append(neighbor)
                visited.append(neighbor)
                
                levels[neighbor] = levels[node] + 1
                
    return target in explored

print(bfs(graph, 'A', 'F'))
print(bfs(graph, 'A', 'G'))

True
False


As shown above, the BFS algorithm correctly outputs that there exists a path between the two vertices *A* and *F*, and it correctly outputs that there exists no path between *A* and *G*. Similarly, we could have used DFS; however, these two algorithms require $\mathsf{\Omega}$(n) space.

## A Second Solution Using a Random Walk

What if we are restricted to logarithmic space? Imagine that we are restricted to only storing a constant number of pointers into the input and a constant number of counters with values in the range [0,n]. In the following, we will see how s-t connectivity has a log-space solution that utilizes randomness.

The algorthm details are shown below:

s-t Connectivity(s,t):
1. Invoke a random walk starting from s.
2. Initialize steps_taken = 0.
3. While the steps_taken < 2n^3:
    4. if t is reached, then RETURN TRUE
5. End-While
6. RETURN FALSE


* **Case 1: There is no path.**
    In this case, the algorithm returns the correct answer, since it will halt after $2n^{3}$ steps either way. Hence, Pr[Algorithm outputs "YES" when real answer is "NO"] = 0 and Pr[Algorithm outputs "NO" when real answer is "NO"] = 1.
    
* **Case 2: There exists an s-t path.** 
    In this case, the algorithm only returns the correct answer if it sees *t* after $2n^{3}$ steps; otherwise, it returns the wrong answer. 
    
    
A key question is how can we bound the probability of failure in the second case. The idea is to note that there exists a path from *s* to *t* iff they are both in the same connected component. This means that the time it takes the random walk to visit *t* when starting from *s* is bounded above by the **cover time** of this connected component. Define the **cover time** of a graph to be the maximum (over all vertices) of the expected time to visit all of the nodes in the graph (at least once) by means of a random walk starting from some vertex [2]. The cover time of a graph is bounded above by 2|E|(|V| - 1), which can be at most $n^{3}$ in the case of a dense graph.

We can bound the probability of failure using Markov's inequality:
$$ \Pr[\text{Algorithm outputs "NO" when real answer is "YES"}] \leq \Pr[\text{cover time} \geq 2n^{3}] \leq \frac{E[X]}{2n^{3}} = \frac{n^{3}}{2n^{3}} = \frac{1}{2} $$

To see this in effect, consider the following tests:

In [40]:
import random 

def random_walk(graph, s, t, max_steps):
    current_vertex = s
    steps_taken = 0
    
    while steps_taken < max_steps:
        if current_vertex == t:
            return True
        steps_taken += 1
        current_vertex = random.choice(graph[current_vertex])
    
    return False

n = len(graph)
print(random_walk(graph, 'A', 'F', n))

False


In [41]:
def test_random_walk():
    # Test cases where there is a path from s to t
    graph_with_path = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B'],
    'D': ['B', 'E'],
    'E': ['D']
    }
    
    # Test cases where there is no path from s to t
    graph_no_path = {
        'A': ['B', 'C'],
        'B': ['A'],
        'C': ['A'],
        'D': ['E'],
        'E': ['D']
    }
    
    for graph in [graph_with_path, graph_no_path]:
        print("Testing graph:", graph)
        n = len(graph)
        for s in graph:
            for t in graph:
                if s != t:
                    result = random_walk(graph, s, t, n)
                    print(f"random_walk({s} -> {t}) : {result}")

test_random_walk()


Testing graph: {'A': ['B', 'C'], 'B': ['A', 'C', 'D'], 'C': ['A', 'B'], 'D': ['B', 'E'], 'E': ['D']}
random_walk(A -> B) : True
random_walk(A -> C) : True
random_walk(A -> D) : False
random_walk(A -> E) : False
random_walk(B -> A) : False
random_walk(B -> C) : True
random_walk(B -> D) : False
random_walk(B -> E) : False
random_walk(C -> A) : True
random_walk(C -> B) : True
random_walk(C -> D) : False
random_walk(C -> E) : False
random_walk(D -> A) : False
random_walk(D -> B) : True
random_walk(D -> C) : True
random_walk(D -> E) : True
random_walk(E -> A) : False
random_walk(E -> B) : True
random_walk(E -> C) : False
random_walk(E -> D) : True
Testing graph: {'A': ['B', 'C'], 'B': ['A'], 'C': ['A'], 'D': ['E'], 'E': ['D']}
random_walk(A -> B) : True
random_walk(A -> C) : True
random_walk(A -> D) : False
random_walk(A -> E) : False
random_walk(B -> A) : True
random_walk(B -> C) : False
random_walk(B -> D) : False
random_walk(B -> E) : False
random_walk(C -> A) : True
random_walk(C -> B) 

In [42]:
def test_bfs():
    # Test cases where there is a path from s to t
    graph_with_path = {
            'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B'],
    'D': ['B', 'E'],
    'E': ['D']
    }
    
    
    # Test cases where there is no path from s to t
    graph_no_path = {
        'A': ['B', 'C'],
        'B': ['A'],
        'C': ['A'],
        'D': ['E'],
        'E': ['D']
    }
    
    for graph in [graph_with_path, graph_no_path]:
        print("Testing graph:", graph)
        n = len(graph)
        for s in graph:
            for t in graph:
                if s != t:
                    result = bfs(graph, s, t)
                    print(f"dfs({s} -> {t}) : {result}")

test_bfs()


Testing graph: {'A': ['B', 'C'], 'B': ['A', 'C', 'D'], 'C': ['A', 'B'], 'D': ['B', 'E'], 'E': ['D']}
dfs(A -> B) : True
dfs(A -> C) : True
dfs(A -> D) : True
dfs(A -> E) : True
dfs(B -> A) : True
dfs(B -> C) : True
dfs(B -> D) : True
dfs(B -> E) : True
dfs(C -> A) : True
dfs(C -> B) : True
dfs(C -> D) : True
dfs(C -> E) : True
dfs(D -> A) : True
dfs(D -> B) : True
dfs(D -> C) : True
dfs(D -> E) : True
dfs(E -> A) : True
dfs(E -> B) : True
dfs(E -> C) : True
dfs(E -> D) : True
Testing graph: {'A': ['B', 'C'], 'B': ['A'], 'C': ['A'], 'D': ['E'], 'E': ['D']}
dfs(A -> B) : True
dfs(A -> C) : True
dfs(A -> D) : False
dfs(A -> E) : False
dfs(B -> A) : True
dfs(B -> C) : True
dfs(B -> D) : False
dfs(B -> E) : False
dfs(C -> A) : True
dfs(C -> B) : True
dfs(C -> D) : False
dfs(C -> E) : False
dfs(D -> A) : False
dfs(D -> B) : False
dfs(D -> C) : False
dfs(D -> E) : True
dfs(E -> A) : False
dfs(E -> B) : False
dfs(E -> C) : False
dfs(E -> D) : True


In the case of a graph where an *s*-*t* path exists, the random walk algorithm fails in 10 out of 20 tests. In the case where there is no path from *s* to *t*, the random walk algorithm always succeeds.

We can run a Monte Carlo simulation to check the proability of failure of the random walk algorithm.

In [48]:
def test_random_walk():
    # Test cases where there is a path from s to t
    graph_with_path = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B'],
    'D': ['B', 'E'],
    'E': ['D']
    }
    
    num_fails = 0
    for graph in [graph_with_path]:
        n = len(graph)
        for s in graph:
            for t in graph:
                if s != t:
                    result = random_walk(graph, s, t, n)
                    if (result is False):
                        num_fails += 1
    return num_fails

# Start the Monte Carlo simulation
N = 100000
sum = 0
for i in range(N):
    x = test_random_walk()
    sum += x
print("A = ", sum/N)

A =  9.12547


As observed, we got 9.1298. This is less than 1/2 when divided by 20.This shows that over many, many runs, the random walk gets at most 1/2 of the answers wrong. 

We can decrease this probability of failure by running the algorithm for $log(n)$ times, bringing down the error probability to $\frac{1}{n}$ [3].

Finally, we can see the sequence of steps taken to reach the target (if the target is ever reached):

In [44]:
def is_connected_random_walk(graph, s, t, max_steps):
    current_vertex = s
    steps_taken = 0
    
    while steps_taken < max_steps:
        print(f"Steps taken: {steps_taken} out of {max_steps}")
        if current_vertex == t:
            return True
        steps_taken += 1
        current_vertex = random.choice(graph[current_vertex])
    
    return False


In [45]:
graph = {
    'A': ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'],
    'B': ['A'],
    'C': ['A'],
    'D': ['A'],
    'E': ['A'],
    'F': ['A'],
    'G': ['A'],
    'H': ['A'],
    'I': ['A'],
    'J': ['A'],
    'K': ['A'],
    'L': ['A'],
    'M': ['A'],
}

In [49]:
is_connected_random_walk(graph, 'A', 'M', steps)

Steps taken: 0 out of 4394
Steps taken: 1 out of 4394
Steps taken: 2 out of 4394
Steps taken: 3 out of 4394
Steps taken: 4 out of 4394
Steps taken: 5 out of 4394
Steps taken: 6 out of 4394
Steps taken: 7 out of 4394
Steps taken: 8 out of 4394
Steps taken: 9 out of 4394
Steps taken: 10 out of 4394
Steps taken: 11 out of 4394
Steps taken: 12 out of 4394
Steps taken: 13 out of 4394
Steps taken: 14 out of 4394
Steps taken: 15 out of 4394
Steps taken: 16 out of 4394
Steps taken: 17 out of 4394
Steps taken: 18 out of 4394
Steps taken: 19 out of 4394
Steps taken: 20 out of 4394
Steps taken: 21 out of 4394
Steps taken: 22 out of 4394
Steps taken: 23 out of 4394
Steps taken: 24 out of 4394
Steps taken: 25 out of 4394
Steps taken: 26 out of 4394
Steps taken: 27 out of 4394
Steps taken: 28 out of 4394
Steps taken: 29 out of 4394
Steps taken: 30 out of 4394
Steps taken: 31 out of 4394
Steps taken: 32 out of 4394
Steps taken: 33 out of 4394
Steps taken: 34 out of 4394
Steps taken: 35 out of 4394
St

True

Different number of steps will be produced for different runs of this code.

## References

[1] *Mitzenmacher, Michael, and Eli Upfal. Probability and computing: Randomization and probabilistic techniques in algorithms and data analysis. Cambridge university press, 2017.*

[2] https://people.csail.mit.edu/ronitt/COURSE/S22/NOTES/lec11-scribe.pdf

[3] https://pages.cs.wisc.edu/~shuchi/courses/787-F09/scribe-notes/lec15.pdf